# Lab 8: Evolutionary computation

### Consider the following example:

Determine the minimum of the function $f(x)= x_1^2+...+x_n^2$ with $x_i \in [-5.12, 5.12]$, $i \in \overline{(1, n)}$

We have an example of steady state genetic algorithm with:  representation an array of real numbers; 100 individuals; crossover $$child = \alpha \cdot (parent1 - parent2) + parent2 ;$$ mutation - reinitialise on a random position the individual's value.

In [17]:


from random import randint, random
from operator import add
from math import cos, pi


def individual(length, vmin, vmax):
    '''
    Create a member of the population - an individual

    length: the number of genes (components)
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    '''
    return [ (random()*(vmax-vmin)+vmin) for x in range(length) ]
    
def population(count, length, vmin, vmax):
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual
    vmin: the minimum possible value 
    vmax: the maximum possible value 
    """
    return [ individual(length, vmin, vmax) for x in range(count) ]

def fitness(individual):
    """
    Determine the fitness of an individual. Lower is better.(min problem)
    For this problem we have the Rastrigin function
    
    individual: the individual to evaluate
    """
    n=len(individual)
    f=0
    for i in range(n):
        f=f+individual[i]*individual[i]
    return f
    
def mutate(individual, pM, vmin, vmax): 
    '''
    Performs a mutation on an individual with the probability of pM.
    If the event will take place, at a random position a new value will be
    generated in the interval [vmin, vmax]

    individual:the individual to be mutated
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    if pM > random():
            p = randint(0, len(individual)-1)
            individual[p] = random()*(vmax-vmin)+vmin
    return individual
    
def crossover(parent1, parent2):
    '''
    crossover between 2 parents
    '''
    child=[]
    alpha=random()
    for x in range(len(parent1)):
        child.append(alpha*(parent1[x]-parent2[x])+parent2[x])
    return child

def iteration(pop, pM, vmin, vmax):
    '''
    an iteration

    pop: the current population
    pM: the probability the mutation to occure
    vmin: the minimum possible value 
    vmax: the maximum possible value
    '''
    i1=randint(0,len(pop)-1)
    i2=randint(0,len(pop)-1)
    if (i1!=i2):
        c=crossover(pop[i1],pop[i2])
        c=mutate(c, pM, vmin, vmax)
        f1=fitness(pop[i1])
        f2=fitness(pop[i2])
        '''
        the repeated evaluation of the parents can be avoided
        if  next to the values stored in the individuals we 
        keep also their fitnesses 
        '''
        fc=fitness(c)
        if(f1>f2) and (f1>fc):
            pop[i1]=c
        if(f2>f1) and (f2>fc):
            pop[i2]=c
    return pop

def main(noIteratii=10000):
    #PARAMETERS:
    
    #population size
    dimPopulation = 100
    #individual size
    dimIndividual = 2
    #the boundries of the search interval
    vmin = -5.12
    vmax = 5.12
    #the mutation probability
    pM=0.01
    
    P = population(dimPopulation, dimIndividual, vmin, vmax)
    for i in range(noIteratii):
        P = iteration(P, pM, vmin, vmax)

    #print the best individual
    graded = [ (fitness(x), x) for x in P]
    graded =  sorted(graded)
    result=graded[0]
    fitnessOptim=result[0]
    individualOptim=result[1]
    print('Result: The detected minimum point after %d iterations is f(%3.2f %3.2f) = %3.2f'% \
          (noIteratii,individualOptim[0],individualOptim[1], fitnessOptim) )
        
main()

Result: The detected minimum point after 10000 iterations is f(-0.00 0.00) = 0.00


Exercise 1:  Construct a similar algorithm to the one provided as an example for the Bukin function N.6 (search the internet for this function).


In [40]:
# f(x) = 100 * sqrt(|x2 - 0.01 * x1^2|) + 0.01 * |x1 + 10|
# Global minimum has been defined as f(-10, 1) = 0
# https://www.sfu.ca/~ssurjano/bukin6.html

from random import randint, random

def individual(x1min, x1max, x2min, x2max):
    return [ (random()*(x1max-x1min)+x1min), (random()*(x2max-x2min)+x2min) ]

def populate(size, x1min, x1max, x2min, x2max):
    return [individual(x1min, x1max, x2min, x2max) for _ in range(size)]

def fitness(individual):
    x1 = individual[0]
    x2 = individual[1]
    return 100 * (x2 - 0.01 * x1**2)**2 + 0.01 * (x1 + 10)**2

def crossover(individual1, individual2):
    child=[]
    alpha=random()
    for x in range(len(individual1)):
        child.append(alpha*(individual1[x]-individual2[x])+individual2[x])
    return child

def mutate(individual, probability, x1min, x1max, x2min, x2max):
    if(probability < random()):
        return individual
    
    p = randint(0, 1)
    if p == 0:
        individual[0] = random()*(x1max-x1min)+x1min
    else:
        individual[1] = random()*(x2max-x2min)+x2min
        
    return individual

def iteration(population, mutationProbability, x1min, x1max, x2min, x2max):
    index1 = randint(0, len(population)-1)
    index2 = randint(0, len(population)-1)

    if index1 != index2:
        child = crossover(population[index1], population[index2])
        child = mutate(child, mutationProbability, x1min, x1max, x2min, x2max)

        fitness1 = fitness(population[index1])
        fitness2 = fitness(population[index2])
        fitnessChild = fitness(child)

        if fitness1 > fitness2 and fitness1 > fitnessChild:
            population[index1] = child
        if fitness2 > fitness1 and fitness2 > fitnessChild:
            population[index2] = child
    
    return population

def main(iterations = 1000):
    populationSize = 100

    searchx1Min = -15
    searchx1Max =  -5

    searchx2Min = -3
    searchx2Max = 3

    mutationProbability = 0.01

    population = populate(populationSize, searchx1Min, searchx1Max, searchx2Min, searchx2Max)

    for i in range(iterations):
        population = iteration(population, mutationProbability, searchx1Min, searchx1Max, searchx2Min, searchx2Max)

    graded = [(fitness(x), x) for x in population]
    graded = sorted(graded)
    result = graded[0]
    fitnessOptim = result[0]
    individualOptim = result[1]

    print('Result: The detected minimum point after %d iterations is f(%3.2f %3.2f) = %3.2f' % \
            (iterations, individualOptim[0], individualOptim[1], fitnessOptim))

main()

Result: The detected minimum point after 1000 iterations is f(-9.99 1.00) = 0.00


Consider the knapsack problem:

Consider a Knapsack with a total volum equal with $V_{max}$.

There are $n$ objects, with values $(p_i)_{n}$ and volumes $(v_i)_n$.

Solve this problem using a generationist Genetic Algorithm, with a binary representation.

Exercise 2: Initialization
Objective: Implement the initialization step of a genetic algorithm.

Exercise 3: Fitness Evaluation

Objective: Implement the fitness evaluation step of a genetic algorithm.

Exercise 4: Selection

Objective: Implement the selection step of a genetic algorithm.

Exercise 5: Crossover

Objective: Implement the crossover step of a genetic algorithm.

Exercise 6: Mutation

Objective: Implement the mutation step of a genetic algorithm.

Exercise 7: Complete Genetic Algorithm

Objective: Combine all the steps of a genetic algorithm to solve a specific problem.

Exercise 8: Extract the result from the final population

Objective: Get the best individual from the final population.


In [123]:
from random import random, randint

def individual(length):
    return [ randint(0,1) for _ in range(length) ]

def populate(count, length):
    return [ individual(length) for _ in range(count) ]

def fitness(individual, objects, max_volume):
    value = 0
    volume = 0
    for i in range(len(individual)):
        if individual[i] == 1:
            value += objects[i][0]
            volume += objects[i][1]
    if volume > max_volume:
        return 0
    return value

def population_fitness(population, objects, max_volume):
    return [ fitness(individual, objects, max_volume) for individual in population ]

def select_parents(population, fitness_scores):
    # select two parents from the population based on the fitness - 
    # the better the fitness, the higher the chance to be selected
    # IN:  population, fitness_scores
    # OUT: selected_parents

    # Sort population depeing on fitness.
    # The higher the fitness, the higher the chance to be selected.
    fitness_sum = sum(fitness_scores) + len(fitness_scores)
    gradedPopulations = [((fitness_scores[index] + 1) / fitness_sum, population[index], index) for index in range(len(population))]
    gradedPopulations = sorted(gradedPopulations, key=lambda x: x[0], reverse=True)

    # Select two parents based on the fitness
    selected_parents = []
    for _ in range(2):
        found = False

        while not found:
            r = random()
            for index in range(len(gradedPopulations)):
                r -= gradedPopulations[index][0]
                if r <= 0 and gradedPopulations[index][2] not in selected_parents:
                    selected_parents.append(gradedPopulations[index][2])
                    found = True
                    break

    return [population[selected_parents[0]], population[selected_parents[1]]]

def crossover(parents):
    # create two new offspring by combining the parents
    # IN:  parents
    # OUT: offspring

    child = []
    for i in range(len(parents[0])):
        if random() > 0.5:
            child.append(parents[0][i])
        else:
            child.append(parents[1][i])

    return child

def mutate(chromosome, mutation_rate):
    # mutate the chromosome by randomly flipping bits
    # IN:  chromosome, mutation_rate
    # OUT: mutated_chromosome

    for index in range(len(chromosome)):
        if random() < mutation_rate:
            chromosome[index] = 1 if chromosome[index] == 0 else 0
    
    return chromosome

# Test everything.
# objects = [(4, 12), (2, 2), (10, 4), (1, 1), (2, 1)]
# max_volume = 15

# population_size = 10

# population = populate(population_size, len(objects))
# print("Population: " + str(population))

# fitness_scores = population_fitness(population, objects, max_volume)
# print("Fitness scores: " + str(fitness_scores))

# parents = select_parents(population, fitness_scores)
# print("Selected Parents: " + str(parents))

# offspring = crossover(parents)
# print(offspring)

# print(fitness(parents[0], objects, max_volume))
# print(fitness(parents[1], objects, max_volume))
# print(fitness(offspring, objects, max_volume))

# mutation_rate = 0.1
# mutated_offspring = mutate(offspring, mutation_rate)
# print(mutated_offspring)
# print(fitness(mutated_offspring, objects, max_volume))


def genetic_algorithm(population_size, generations, mutation_rate, objects, max_volume):
    # complete genetic algorithm
    # IN:  population_size, generations, mutation_rate, objects, max_volume
    # OUT: population

    # initialize the population
    population = populate(population_size, len(objects))

    for index in range(generations):
        print("Generation " + str(index + 1))
        print("Population " + str(population))
        print("Fitness scores " + str(population_fitness(population, objects, max_volume)))
        newPopulation = []

        # Fitness evaluation
        fitness_scores = population_fitness(population, objects, max_volume)

        for _ in range(population_size):
            # Selection
            parents = select_parents(population, fitness_scores)

            # Crossover
            offspring = crossover(parents)

            # Mutation
            offspring = mutate(offspring, mutation_rate)

            # Replace the population with the new generation
            newPopulation.append(offspring)

        population = newPopulation

    return population

# # Test the complete genetic algorithm
population_size = 10
generations = 100
mutation_rate = 0.1
objects = [(3, 2), (5, 4), (8, 5), (4, 3), (10, 9)]
max_volume = 20

final_population = genetic_algorithm(population_size, generations, mutation_rate, objects, max_volume)
print("Final population: ")
print(final_population)

best_individual = max(final_population, key=lambda x: fitness(x, objects, max_volume))
print("Best individual: ")
print(best_individual)

print("Best fitness:")
print(fitness(best_individual, objects, max_volume))



Generation 1
Population [[1, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1]]
Fitness scores [11, 18, 11, 5, 18, 10, 4, 13, 16, 15]
Generation 2
Population [[0, 1, 0, 0, 1], [1, 1, 0, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 0, 0], [0, 0, 0, 0, 1]]
Fitness scores [15, 18, 26, 16, 0, 15, 11, 23, 13, 10]
Generation 3
Population [[1, 1, 1, 0, 1], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]
Fitness scores [26, 16, 0, 16, 26, 13, 23, 8, 23, 22]
Generation 4
Population [[0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 0, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [0, 1, 1, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1]]
Fitness scores [8, 18, 26, 15, 19, 15, 23, 26, 16, 0]
Gen